<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png"
     width="200px"
     height="auto"/>
</p>

# <h1 align="center" id="heading">Ecommerce Recommender via Graph Neural Network</h1>

# Part 0: Config

In [1]:
import yaml
import pandas as pd
import glob

In [2]:
%cd ..

/Users/yingkang/4thBrain/GNN-eCommerce


In [3]:
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)
config

{'base': {'random_seed': 42},
 'data': {'cosmetic_shop': 'data/raw/cosmetic-shop-ecommerce-events/'},
 'training': None,
 'reports': None}

# Part 1: EDA



## 1.0: Data Loader


In [4]:
# Get CSV files list of cosmetic shop dataset(month data per file)
path = config['data']['cosmetic_shop']
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame. This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
all_events = pd.concat(df_list, ignore_index=True)


In [5]:
all_events.shape

(20692840, 9)

In [6]:
all_events.sample(10)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
18917661,2019-10-17 17:28:07 UTC,cart,5851067,1487580013841613016,NaN,kaypro,12.06,561304347,07d23998-5249-476d-996d-232f71b2dfb8
17584087,2019-10-07 14:35:17 UTC,view,5001,1487580010721050707,NaN,NaN,20.63,554689425,f88650d2-b0f3-4c08-a0c4-4e4b243beb5b
14594764,2019-11-20 11:50:43 UTC,view,5842153,1487580006317032337,NaN,NaN,1.59,533413947,3cda2b17-19b8-4613-b17f-a863a551eb63
4396352,2019-12-02 03:31:38 UTC,remove_from_cart,5779268,1487580006962955182,NaN,NaN,7.14,580614449,fe9fd180-49c4-4133-b701-f0985d115586
14488242,2019-11-19 16:37:08 UTC,remove_from_cart,5820749,1487580006317032337,NaN,NaN,7.14,416344202,44b840d9-bef2-40b1-bef1-0fd8b1422cef
11984446,2019-11-01 09:05:52 UTC,cart,5793261,1487580010100293687,NaN,NaN,22.21,371505220,38aa8289-e5a2-44f0-bdc7-e0018a4e6a60
8579517,2020-02-05 20:11:31 UTC,remove_from_cart,5800788,1487580007256556476,NaN,NaN,4.52,562386185,8c70b883-9a29-4a6d-a40e-043746166c75
5252016,2019-12-08 18:59:34 UTC,view,5809910,1602943681873052386,NaN,grattol,5.24,583962448,74da65cb-58ac-4f8d-a8de-d67ef7150c41
16977373,2019-10-03 10:28:07 UTC,remove_from_cart,5800788,1487580007256556476,NaN,NaN,4.52,513406182,0d0aa2c8-fd43-48e3-812f-a1bf4b9a0c4e
16526924,2019-11-30 16:14:10 UTC,remove_from_cart,4958,1487580009471148064,NaN,runail,1.10,550575227,b9e94d91-92dc-4880-9cac-1862f5920a0d


In [7]:
# persist all_events to a csv file
all_events.to_csv('data/raw/cosmetic-shop-ecommerce-events/combined_raw.csv')

## 1.1. Explore, Describe, Counts


In [8]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

In [11]:
# read from persisted csv file
df = pd.read_csv('data/raw/cosmetic-shop-ecommerce-events/combined_raw.csv')
df.shape

(20692840, 10)

In [12]:
df.sample(10)

,Unnamed: 0,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
799567,799567,2020-01-08 09:37:11 UTC,view,5345,1487580009445982239,NaN,runail,1.27,533047380,9907b120-e59d-4f84-b993-d690d947b652
2088368,2088368,2020-01-17 02:47:37 UTC,cart,5374,1487580009496313889,NaN,runail,6.83,576458292,38389f30-35db-4675-b4e1-ec69674d954e
13214559,13214559,2019-11-09 21:44:08 UTC,purchase,5881332,1487580005092295511,NaN,beautix,15.71,442585177,e1c3bf27-93d3-4bf2-a857-ddcd46e5a39d
13030437,13030437,2019-11-08 14:09:02 UTC,remove_from_cart,5773607,1487580006317032337,NaN,irisk,1.59,434872123,11803d3a-c1f2-436b-a311-05f92aa55178
3880500,3880500,2020-01-29 09:19:07 UTC,cart,5768057,1487580012574933146,NaN,NaN,4.05,406615497,75ed5209-82aa-4669-8bbb-966f4397f8d4
17659446,17659446,2019-10-08 05:11:04 UTC,cart,5808322,1487580009362096156,NaN,kapous,1.95,557808874,b7542cfb-20eb-4a13-b83d-0693a8640f25
4691121,4691121,2019-12-04 08:10:34 UTC,view,5759979,1487580009051717646,NaN,irisk,51.59,581848711,21e0c137-8e26-4f1d-8dd7-724520726b9a
14892149,14892149,2019-11-21 22:24:00 UTC,remove_from_cart,5854341,1487580005092295511,NaN,jessnail,2.19,501711265,6c48be86-84bb-4084-b782-814135cc5919
14204187,14204187,2019-11-17 17:49:14 UTC,view,5773329,1487580005134238553,NaN,runail,2.62,485440188,b993adbc-36f9-493a-9fde-eddd917f0616
5640294,5640294,2019-12-11 11:48:21 UTC,remove_from_cart,5779813,1638456119066100510,NaN,pole,4.11,515577705,304a4d3e-308d-482f-b01b-ae9fa0fb0c3c


In [29]:
# Number of unique users
users_count = len(pd.unique(df['user_id']))
print("Unique users count: ", users_count)

# Number of unique products
products_count = len(pd.unique(df['product_id']))
print("Unique products count: ", products_count)

# Number of unique event_type
event_type_count = len(pd.unique(df['event_type']))
print("Unique event type count: ", event_type_count)

Unique users count:  1639358
Unique products count:  54571
Unique event type count:  4


In [14]:
print('all_events.isna().sum():')
df.isna().sum()

all_events.isna().sum():


Unnamed: 0              0
event_time              0
event_type              0
product_id              0
category_id             0
category_code    20339246
brand             8757117
price                   0
user_id                 0
user_session         4598
dtype: int64

### check NaN brand and category_code vs category_id

### check # of session per user

In [16]:
session_by_user = df.groupby("user_id")["user_session"].count()

In [21]:
session_by_user.head(10)

user_id
465496        3
1120748       1
1180452       1
1458813      31
2038666       1
             ..
622090016     2
622090043     3
622090052     2
622090098     1
622090237     1
Name: user_session, Length: 1639358, dtype: int64

### check # of events(view/ add/ remove/ buy) per user

In [27]:
events_per_user = df.groupby(["user_id", "event_type"], as_index=True)["event_type"].count()

In [28]:
events_per_user

user_id    event_type
465496     view           3
1120748    view           1
1180452    view           1
1458813    view          31
2038666    view           1
                         ..
622090043  cart           1
           view           2
622090052  view           2
622090098  view           1
622090237  view           1
Name: event_type, Length: 2289812, dtype: int64

### check price range per user, mark buy price within the range